In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Лабораторная 2 — Сбор и подготовка данных\n",
    "\n",
    "Цель: подготовить корпус отзывов RuReviews, провести очистку, EDA и сформировать выборку для разметки."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "!pip install pandas tqdm langdetect regex matplotlib seaborn pyarrow fastparquet"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, regex as re, json\n",
    "from tqdm.auto import tqdm\n",
    "from langdetect import detect\n",
    "from pathlib import Path\n",
    "tqdm.pandas()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Загрузка датасета RuReviews"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "csv_path = Path('../rureviews/women-clothing-accessories.3-class.balanced.csv')\n",
    "df = pd.read_csv(csv_path)\n",
    "df.head()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Унификация колонок\n",
    "rename_map = {'text': 'raw_text', 'score': 'rating'}\n",
    "df = df.rename(columns=rename_map)\n",
    "df['raw_text'] = df['raw_text'].astype(str)"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Базовая статистика"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df['char_len'] = df['raw_text'].str.len()\n",
    "df['word_count'] = df['raw_text'].str.split().str.len()\n",
    "print(df['rating'].value_counts(normalize=True))\n",
    "df[['char_len','word_count']].describe()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "sns.histplot(df['word_count'], bins=50)\n",
    "plt.title('Распределение длины отзывов'); plt.show()\n",
    "sns.countplot(x='rating', data=df); plt.title('Баланс классов'); plt.show()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Очистка текста"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "URL_RE = re.compile(r'https?://\\S+|www\\.\\S+')\n",
    "NON_PRINT_RE = re.compile(r'[\\x00-\\x1f\\x7f-\\x9f]')\n",
    "EMOJI_RE = re.compile(r'\\p{So}+', re.UNICODE)\n",
    "MULTI_RE = re.compile(r'\\s+')\n",
    "\n",
    "def detect_lang_safe(t):\n",
    "    try: return detect(t)\n",
    "    except: return 'unknown'\n",
    "\n",
    "def clean_text(t):\n",
    "    s = str(t)\n",
    "    s = URL_RE.sub(' ', s)\n",
    "    s = NON_PRINT_RE.sub(' ', s)\n",
    "    s = EMOJI_RE.sub(' ', s)\n",
    "    s = re.sub(r'[^\\p{L}\\p{N}\\s\\.,!?;:()-]', ' ', s)\n",
    "    s = MULTI_RE.sub(' ', s).strip()\n",
    "    return s.lower()\n",
    "\n",
    "df['lang'] = df['raw_text'].progress_apply(detect_lang_safe)\n",
    "df = df[df['lang'].isin(['ru','unknown'])]\n",
    "df['clean_text'] = df['raw_text'].progress_apply(clean_text)"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Удаление дублей и коротких отзывов"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df = df.drop_duplicates(subset=['clean_text'])\n",
    "df = df[df['char_len']>=15]\n",
    "df.shape"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Подготовка sample для аннотации и экспорт"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "OUT = Path('../data/processed'); OUT.mkdir(parents=True, exist_ok=True)\n",
    "sample = df.groupby('rating', group_keys=False).apply(lambda x: x.sample(min(len(x),333), random_state=42))\n",
    "sample.to_csv(OUT/'annotation_sample.csv', index=False)\n",
    "df[['clean_text','rating','lang','word_count','char_len']].to_parquet(OUT/'reviews_processed.parquet', index=False)"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Quality Report"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "report = {\n",
    "  'n_total': int(len(df)),\n",
    "  'lang_dist': df['lang'].value_counts().to_dict(),\n",
    "  'avg_words': float(df['word_count'].mean()),\n",
    "  'rating_dist': df['rating'].value_counts().to_dict()\n",
    "}\n",
    "Path('../reports').mkdir(parents=True, exist_ok=True)\n",
    "with open('../reports/quality_report.json','w',encoding='utf-8') as f:\n",
    "    json.dump(report,f,ensure_ascii=False,indent=2)\n",
    "report"
   ],
   "execution_count": null,
   "outputs": []
  }
 ],
 "metadata": {
  "kernelspec": {"name": "python3"},
  "language_info": {"name": "python"}
 },
 "nbformat": 4,
 "nbformat_minor": 2
}